In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn

# torch.seed(42)

# Dataset Preprocessing

Split data into train, validation and test:  
80% train, 10% validation, 10% test:

Years selected for test and validation (based of random number generation):
Valid and test years pairs are selected within one year of seperation
valid = 2022, 1999, 2008, 1990
test = 2023,  1998, 2009, 1991

Validation years

In [10]:
datafolder = "./data/"
merge_df = pd.DataFrame()
for i, file in enumerate(os.listdir(datafolder)):
    if i == 0:
        merge_df = pd.read_csv(datafolder + file)
        print(type(merge_df))
    else:
        df = pd.read_csv(datafolder + file)
        merge_df = pd.merge(merge_df, df, on="month")

merge_df["month_num"] = merge_df["month"].apply(lambda x: int(x.split("-")[1]))

merge_df


<class 'pandas.core.frame.DataFrame'>


,month,maximum_rainfall_in_a_day,no_of_rainy_days,total_rainfall,rh_extremes_minimum,mean_rh,mean_sunshine_hrs,mean_temp,temp_mean_daily_max,temp_mean_daily_min,month_num
0,1982-01,36.5,10,107.1,49,81.2,5.6,25.9,29.8,23.3,1
1,1982-02,9.4,5,27.8,49,79.5,7.6,27.1,32.3,23.9,2
2,1982-03,61.7,11,160.8,51,82.3,5.1,27.2,31.7,24.3,3
3,1982-04,45.1,14,157.0,54,85.9,4.7,27.0,31.4,24.3,4
4,1982-05,33.0,10,102.2,57,83.2,5.8,28.0,31.7,25.2,5
...,...,...,...,...,...,...,...,...,...,...,...
513,2024-10,66.0,15,147.6,41,78.9,5.0,28.5,32.1,25.8,10
514,2024-11,71.2,24,407.4,53,84.5,4.3,27.4,31.6,25.0,11
515,2024-12,76.8,23,234.4,41,80.7,4.6,27.7,31.7,25.3,12
516,2025-01,157.0,18,480.2,54,83.4,3.6,26.5,29.5,24.6,1


In [ ]:
group_2 = ['2022', '1999', '2008', '1990']
group_3 = ['2023', '2009', '1998', '1991']

# Filter DataFrames by year
valid_df = merge_df[merge_df['month'].str.split('-').str[0].isin(group_2)]
test_df = merge_df[merge_df['month'].str.split('-').str[0].isin(group_3)]

train_df = merge_df[~merge_df['month'].str.split('-').str[0].isin(group_2 + group_3)]

In [19]:
train_df.month.unique()

array(['1982-01', '1982-02', '1982-03', '1982-04', '1982-05', '1982-06',
       '1982-07', '1982-08', '1982-09', '1982-10', '1982-11', '1982-12',
       '1983-01', '1983-02', '1983-03', '1983-04', '1983-05', '1983-06',
       '1983-07', '1983-08', '1983-09', '1983-10', '1983-11', '1983-12',
       '1984-01', '1984-02', '1984-03', '1984-04', '1984-05', '1984-06',
       '1984-07', '1984-08', '1984-09', '1984-10', '1984-11', '1984-12',
       '1985-01', '1985-02', '1985-03', '1985-04', '1985-05', '1985-06',
       '1985-07', '1985-08', '1985-09', '1985-10', '1985-11', '1985-12',
       '1986-01', '1986-02', '1986-03', '1986-04', '1986-05', '1986-06',
       '1986-07', '1986-08', '1986-09', '1986-10', '1986-11', '1986-12',
       '1987-01', '1987-02', '1987-03', '1987-04', '1987-05', '1987-06',
       '1987-07', '1987-08', '1987-09', '1987-10', '1987-11', '1987-12',
       '1988-01', '1988-02', '1988-03', '1988-04', '1988-05', '1988-06',
       '1988-07', '1988-08', '1988-09', '1988-10', 

In [18]:
valid_df.month.unique()

array(['1990-01', '1990-02', '1990-03', '1990-04', '1990-05', '1990-06',
       '1990-07', '1990-08', '1990-09', '1990-10', '1990-11', '1990-12',
       '1999-01', '1999-02', '1999-03', '1999-04', '1999-05', '1999-06',
       '1999-07', '1999-08', '1999-09', '1999-10', '1999-11', '1999-12',
       '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06',
       '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12',
       '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12'],
      dtype=object)

In [17]:
test_df.month.unique()

array(['1991-01', '1991-02', '1991-03', '1991-04', '1991-05', '1991-06',
       '1991-07', '1991-08', '1991-09', '1991-10', '1991-11', '1991-12',
       '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06',
       '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12',
       '2009-01', '2009-02', '2009-03', '2009-04', '2009-05', '2009-06',
       '2009-07', '2009-08', '2009-09', '2009-10', '2009-11', '2009-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
       '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12'],
      dtype=object)

In [20]:
# normalize data except for month and predicting value which is total_rainfall

# Either Gaussian Normalization or Min-Max Normalization

def normalised(df):
    predicting_column = "total_rainfall"

    excluded_columns = ["month", predicting_column]
    # print(merge_df.columns)

    #reorder the columns so that predicting value is at the end
    cols = [col for col in df.columns if col != predicting_column]  # Get all columns except "total_rainfall"
    # print(cols)
    cols.append(predicting_column)  # Add "total_rainfall" at the end
    df = df[cols]

    merge_df_normalized_gaussian = df.copy()

    epsilon = 1e-10

    for column in merge_df_normalized_gaussian.columns:
        if column not in excluded_columns:
            merge_df_normalized_gaussian[column] = (merge_df_normalized_gaussian[column] - merge_df_normalized_gaussian[column].mean()) / merge_df_normalized_gaussian[column].std()
            
    merge_df_normalized_minmax = merge_df.copy()

    for column in merge_df_normalized_minmax.columns:
        if column not in excluded_columns:
            merge_df_normalized_minmax[column] = (merge_df_normalized_minmax[column] - merge_df_normalized_minmax[column].min() + epsilon) / (merge_df_normalized_minmax[column].max() - merge_df_normalized_minmax[column].min() + epsilon)

    return merge_df_normalized_minmax

normalised_train_df = normalised(train_df)
normalised_valid_df = normalised(valid_df)
normalised_test_df = normalised(test_df)

# ModeL with RNN